# 전체 classification 결과를 환자 단위, 파트 별 단위로 묶어서 결과 만드는 코드

In [ ]:
import os
import numpy as np
import pandas

## ENV SETTING

In [ ]:
label_type = '3classes'
learning_rate = '5e-5'

# true label env
true_dataset_root = 'E:/Thesis_research/Database/Medical/Dental_directory_dataset'
true_lbl_dir = os.path.join(true_dataset_root, 'ClassificationClass',label_type)

# prediction env
pred_root = f'E:/Thesis_research/results_materials/Dental/raw_prediction_results/{learning_rate}'

## PREDICTION SETTING AND VOTING

* 각 네트워크 별로 4개의 part에 대한 prediction 중 unique 병록번호에 해당하는 prediction들을 모아서 voting해서 true와 비교!

In [ ]:
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

part_list = [16, 26, 36, 46]
exp_dir = os.path.join(pred_root, label_type)
fold_name_list = sorted(os.listdir(exp_dir))

## Overall accuracy metric
patient_wise_overall_acc_lst = []
part_wise_overall_acc_dic = {}
for i_part in part_list:
    part_wise_overall_acc_dic[i_part] = []

## Confusion matrix metric
confusion_matrix_metric_tot_lst = []
confusion_matrix_metric_partwise_tot_dict = {}
for i_part in part_list:
    confusion_matrix_metric_partwise_tot_dict[i_part] = []
    
for i_fold_iter, i_fold_name in enumerate(fold_name_list):
    print()
    print('Current: ', i_fold_name)
    
    ## TRUE LABEL SETTING

    # * Construct (personnum,true label) pair
    true_imageset_path = os.path.join(true_dataset_root,'ImageSets','Classification','eval' + str(i_fold_iter+1) + '.txt')
    
    with open(true_imageset_path, 'r') as f:
         eval_img_list = f.read().split('\n')

    person_num_list =[]
    for i_eval_img in eval_img_list:
        if i_eval_img == '':
            continue
        eval_img_info = i_eval_img.split('_')
        age_person_num = eval_img_info[0] + '_' + eval_img_info[1] # e.g. '20_2392392' because there are three miss labeled images file name
        if len(eval_img_info)>1: # skip blank line
            person_num_list.append(age_person_num)
    person_num_unique_list, unique_idx = np.unique(np.array(person_num_list), return_index=True)
    
    pred_by_part = {}
    true_by_part = {}
    for i_part in part_list:
        pred_by_part[i_part] = []
        true_by_part[i_part] = []
    
    person_num_perdiction_all_list = []
    true_lbl_unique = []
    
    for i_iter, i_person_num_unique in enumerate(person_num_unique_list):
        pred_dir = os.path.join(exp_dir, i_fold_name, 'eval_result_resnet152_cls_best_model', 'prediction_class')
        pred_result_list = sorted(os.listdir(pred_dir))
        pred_result_person_num = [s for s in pred_result_list if i_person_num_unique in s]
        
        # 하나라도 파트 없으면 false alarm!!
        if not len(pred_result_person_num) == 4 :
            print('Each person must have four teeth parts')
            raise AssertionError
            
        # true label setting
        true_lbl = 0
        for i, i_pred in enumerate(pred_result_person_num):
            true_lbl_path = os.path.join(true_lbl_dir, i_pred)
            with open(true_lbl_path,'r') as f:
                lbl = int(f.read())
            if i==0:
                true_lbl = lbl
            else:
                if true_lbl != lbl: # check all patients label is the same each other
                    raise AssertionError
                else:
                    true_lbl = lbl
        true_lbl_unique.append(true_lbl)
            
        person_num_prediction = []
        for i_pred in pred_result_person_num:
            pred_txt_nameOnly = os.path.splitext(i_pred)[0]
            pred_name_info = pred_txt_nameOnly.split('_')
            part_num = int(pred_name_info[-1])
            pred_result_path = os.path.join(pred_dir, i_pred)
            with open(pred_result_path, 'r') as f:
                pred_lbl = int(f.read())
            person_num_prediction.append(pred_lbl)
            pred_by_part[part_num].append(pred_lbl)
            true_by_part[part_num].append(true_lbl)
            
            
        person_num_perdiction_all_list.append(person_num_prediction)
    
    network_final_pred_list = []
    for i_person_num_pred in person_num_perdiction_all_list:
        most_common_pred, num_most_common_pred = Counter(i_person_num_pred).most_common(1)[0] # 4, 6 times
        network_final_pred_list.append(most_common_pred)
    
    confusion_matrix_metric = confusion_matrix(true_lbl_unique, network_final_pred_list)
    print('Confusion matrix: ')
    print(confusion_matrix_metric)
    confusion_matrix_metric_tot_lst.append(confusion_matrix_metric)
    
    overall_acc_metric = accuracy_score(true_lbl_unique, network_final_pred_list)
    print('Overall accuracy = ', overall_acc_metric)
    
    patient_wise_overall_acc_lst.append(overall_acc_metric)
    
    
    ## save as excel
    index =['True'+str(i) for i in range(1, confusion_matrix_metric.shape[0]+1)]
    columns = ['Pred'+str(i) for i in range(1, confusion_matrix_metric.shape[0]+1)]
    df_total = pandas.DataFrame(confusion_matrix_metric, index=index, columns = columns)
    save_excel_path = os.path.join(exp_dir, i_fold_name, 'conf_mat_total.xlsx')
    df_total.to_excel(save_excel_path)
    
    
    print('======== part by part metric ==========')
    for key in pred_by_part:
        print('PART NUM: ', key)
        confusion_matrix_metric = confusion_matrix(true_by_part[key], pred_by_part[key])
        confusion_matrix_metric_partwise_tot_dict[key].append(confusion_matrix_metric) 
        print('Confusion matrix: ')
        print(confusion_matrix_metric)
        overall_acc_metric = accuracy_score(true_by_part[key], pred_by_part[key])
        print('Overall accuracy = ', overall_acc_metric)
        part_wise_overall_acc_dic[key].append(overall_acc_metric)
        
        index =['True'+str(i) for i in range(1, confusion_matrix_metric.shape[0]+1)]
        columns = ['Pred'+str(i) for i in range(1, confusion_matrix_metric.shape[0]+1)]
        df_part = pandas.DataFrame(confusion_matrix_metric, index=index, columns = columns)
        save_excel_path = os.path.join(exp_dir, i_fold_name, 'conf_mat_part' + str(key) +'.xlsx')
        df_total.to_excel(save_excel_path)

confmat_save_path = os.path.join(exp_dir, '3cls_conf_mat_tot.npy')
np.save(confmat_save_path, confusion_matrix_metric_tot_lst)


# Patient wise cv 결과 정리

In [ ]:
print('Confusion matrix: ')
confusion_matrix_metric_tot = np.array(confusion_matrix_metric_tot_lst)
confusion_matrix_metric_avg = np.mean(confusion_matrix_metric_tot, axis = 0)
print(confusion_matrix_metric_avg)
print()
print('Overall Accuracy: ')
patient_wise_avg_acc = np.mean(patient_wise_overall_acc_lst)
patient_wise_std_error= np.std(patient_wise_overall_acc_lst) / np.sqrt(len(patient_wise_overall_acc_lst))
print('acc: ',patient_wise_avg_acc)
print('std_error: ', patient_wise_std_error)
print()
print('Group-wise accuracy: ')
group_wise_acc_dict={}
for i_group in range(confusion_matrix_metric_tot.shape[1]):
    group_wise_acc_dict[i_group] = []
    for i_fold in range(confusion_matrix_metric_tot.shape[0]):
        confusion_matrix_cur = confusion_matrix_metric_tot[i_fold]
        group_wise_acc = confusion_matrix_cur[i_group, i_group] / np.sum(confusion_matrix_cur[i_group, :])
        group_wise_acc_dict[i_group].append(group_wise_acc)
        
    group_wise_acc_mean = np.mean(group_wise_acc_dict[i_group])
    group_wise_acc_std_error = np.std(group_wise_acc_dict[i_group]) / np.sqrt(len(group_wise_acc_dict[i_group]))
    print('Age group ' + str(i_group+1))
    print('acc: ',group_wise_acc_mean)
    print('std_error: ',group_wise_acc_std_error)
    print()


In [ ]:
len(confusion_matrix_metric_tot_lst)

# Part wise cv 결과 정리

In [ ]:
for i_part in part_list:
    print('===============')
    print('Part', i_part)

    print('Confusion matrix: ')
    confusion_matrix_metric_tot = np.array(confusion_matrix_metric_partwise_tot_dict[i_part])
    confusion_matrix_metric_avg = np.mean(confusion_matrix_metric_tot, axis = 0)
    print(confusion_matrix_metric_avg)
    print()
    print('Overall Accuracy: ')
    part_wise_avg_acc = np.mean(part_wise_overall_acc_dic[i_part])
    part_wise_std_error= np.std(part_wise_overall_acc_dic[i_part]) / np.sqrt(len(part_wise_overall_acc_dic[i_part]))
    print('acc: ', part_wise_avg_acc)
    print('std_error: ', part_wise_std_error)
    print()



## Patient-wise, part-wise 비교

In [ ]:
print('==== Patient-wise overall acc ====')
patient_wise_avg_acc = np.mean(patient_wise_overall_acc_lst)
patient_wise_std_error= np.std(patient_wise_overall_acc_lst) / np.sqrt(len(patient_wise_overall_acc_lst))
print('avg: ',patient_wise_avg_acc * 100)
print('std_error: ', patient_wise_std_error*100)

print('==== Part-wise overall acc ====')
for i_part in part_list:
    print('Part', i_part)
    part_wise_avg_acc = np.mean(part_wise_overall_acc_dic[i_part])
    part_wise_std_error= np.std(part_wise_overall_acc_dic[i_part]) / np.sqrt(len(part_wise_overall_acc_dic[i_part]))
    print('avg: ', part_wise_avg_acc*100)
    print('std_error: ', part_wise_std_error*100)
    print()

In [ ]:
## save data for using at t-test in 3cls+5cls voting
np.save(os.path.join(exp_dir, '3cls_patient_wise_acc_lst.npy'), patient_wise_overall_acc_lst)
for i_part in part_list:
    out_file_name = os.path.join(exp_dir, '3cls_part'+str(i_part)+'_acc_lst.npy')
    np.save(out_file_name, part_wise_overall_acc_dic[i_part])


In [ ]:
from scipy.stats import ttest_ind
for i_part in part_list:
    print('Patient-wise acc vs part ' + str(i_part) + ' acc')
    ttest,pval = ttest_ind(patient_wise_overall_acc_lst,part_wise_overall_acc_dic[i_part])
    print("p-value", pval)
    print()

In [ ]:
from scipy.stats import ttest_ind
for i_part in part_list:
    print('Patient-wise acc vs part ' + str(i_part) + ' acc')
    ttest,pval = ttest_ind(patient_wise_overall_acc_lst,part_wise_overall_acc_dic[i_part], equal_var=False)
    print("p-value",pval)
    print()